In [2]:
import tensorflow as tf

In [3]:
size = 5
data = list(range(10))
print(data)     # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
X=[]
for i in range(len(data)):
    _X = data[i:size+ i]
    X.insert(i,_X)
print(X)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[[0, 1, 2, 3, 4], [1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8], [5, 6, 7, 8, 9], [6, 7, 8, 9], [7, 8, 9], [8, 9], [9]]


## range() / window()

In [20]:
dataset = tf.data.Dataset.range(10) 
# print(type(dataset))
# for val in dataset:
#     print(val.numpy())

# dataset = dataset.window(5, shift=1) 
dataset = dataset.window(5, shift=1,drop_remainder=True)  # 나머지는 제거
print(type(dataset))
for window_dataset in dataset:     
    for val in window_dataset:   
        print(val.numpy(), end=",")
    print()

<class 'tensorflow.python.data.ops.window_op._WindowDataset'>
0,1,2,3,4,
1,2,3,4,5,
2,3,4,5,6,
3,4,5,6,7,
4,5,6,7,8,
5,6,7,8,9,


2024-07-24 02:03:46.135665: W tensorflow/core/framework/dataset.cc:959] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


## batch()

In [38]:
dataset = tf.data.Dataset.range(100)
dataset = dataset.batch(10)
print(type(dataset))
for val in dataset:
    print(val.numpy())

<class 'tensorflow.python.data.ops.batch_op._BatchDataset'>
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]
[40 41 42 43 44 45 46 47 48 49]
[50 51 52 53 54 55 56 57 58 59]
[60 61 62 63 64 65 66 67 68 69]
[70 71 72 73 74 75 76 77 78 79]
[80 81 82 83 84 85 86 87 88 89]
[90 91 92 93 94 95 96 97 98 99]


[<tf.Tensor: shape=(3,), dtype=int64, numpy=array([0, 1, 2])>,
 <tf.Tensor: shape=(3,), dtype=int64, numpy=array([3, 4, 5])>,
 <tf.Tensor: shape=(3,), dtype=int64, numpy=array([6, 7, 8])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([9])>]

## flat_map()

In [51]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
# print(dataset)
for window_dataset in dataset:     
    for val in window_dataset:   
        print(val.numpy(), end=",")
    print()
    
print('-'*50)
dataset = dataset.flat_map(lambda window: window.batch(5))
for window in dataset:
    print(window.numpy())
    print('-'*50)
    # input()

0,1,2,3,4,
1,2,3,4,5,
2,3,4,5,6,
3,4,5,6,7,
4,5,6,7,8,
5,6,7,8,9,
--------------------------------------------------
[0 1 2 3 4]
--------------------------------------------------
[1 2 3 4 5]
--------------------------------------------------
[2 3 4 5 6]
--------------------------------------------------
[3 4 5 6 7]
--------------------------------------------------
[4 5 6 7 8]
--------------------------------------------------
[5 6 7 8 9]
--------------------------------------------------


2024-07-24 04:17:28.938914: W tensorflow/core/framework/dataset.cc:959] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


## map()

In [50]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
for x,y in dataset:
    print(x.numpy(), y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


## shuffle()

In [64]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
for x,y in dataset:
    print(x.numpy(), y.numpy())

[3 4 5 6] [7]
[0 1 2 3] [4]
[5 6 7 8] [9]
[4 5 6 7] [8]
[1 2 3 4] [5]
[2 3 4 5] [6]


## prefetch()

In [70]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1)
# prefetch(1)을 호출하면 데이터셋은 항상 한 배치가 미리 준비되도록한다 
for x,y in dataset:
    print("x = ", x.numpy())
    print("y = ", y.numpy())

[[3 4 5 6]
 [2 3 4 5]] , [[7]
 [6]]
[[4 5 6 7]
 [0 1 2 3]] , [[8]
 [4]]
[[5 6 7 8]
 [1 2 3 4]] , [[9]
 [5]]


## from_tensor_slices()

In [76]:
# Slicing a 1D tensor produces scalar tensor elements.
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
list(dataset.as_numpy_iterator())

[1, 2, 3]

In [77]:
# Slicing a 2D tensor produces 1D tensor elements.
dataset = tf.data.Dataset.from_tensor_slices([[1, 2], [3, 4]])
list(dataset.as_numpy_iterator())

[array([1, 2], dtype=int32), array([3, 4], dtype=int32)]

In [80]:
# Slicing a tuple of 1D tensors produces tuple elements containing
# scalar tensors.
dataset = tf.data.Dataset.from_tensor_slices(([1, 2], [3, 4], [5, 6]))
list(dataset.as_numpy_iterator())

[(1, 3, 5), (2, 4, 6)]

In [82]:
# Dictionary structure is also preserved.
dataset = tf.data.Dataset.from_tensor_slices({"a": [1, 2], "b": [3, 4]})
list(dataset.as_numpy_iterator())

[{'a': 1, 'b': 3}, {'a': 2, 'b': 4}]